# Explorative Data Analysis - Patients

In this notebook you find the explorative data analysis for the data of patients.
There is also diagnostic data of the patients, data of the used MRI machines and there is also laboratory data available.


## Imports and Preprocessing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# set default plt figsize to (12,6)
plt.rcParams["figure.figsize"] = (12,6)
pd.set_option('display.colheader_justify', 'center')
# Set default font sizes
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18 

# plt.rcParams['font.size'] = 20 
# plt.rcParams['axes.suptitlefontsize'] = 20
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['legend.title_fontsize'] = 18


import os
print(os.getcwd())
if os.getcwd().endswith("eda"):
    os.chdir("..")
    from src.imputation import impute_dataframe,get_imputation_values
    os.chdir("eda")


In [ ]:
# # runs the clean and preprocessing notebook
%run "clean_preprocessing.ipynb"

In [ ]:
# # runs the data_partitioning notebook
%run "data_partitioning.ipynb"

In [ ]:
# read train data set
df = pd.read_csv(r'../data/train/train_lab_data.csv')

In [ ]:
df = impute_dataframe(df,get_imputation_values(df))

In [ ]:
df.loc[df['Category'] == 'prolaktinom','Category'] = 'prolactinoma'
df.loc[df['Category'] == 'non-prolaktinom','Category'] = 'non-prolactinoma'

In [ ]:
# make datetime values
df["Date_Case"] = pd.to_datetime(df["Date_Case"])
df["Diagnosis_Date"] = pd.to_datetime(df["Diagnosis_Date"])
df["Operation_date"] = pd.to_datetime(df["Operation_date"])
# set category data type in pandas, check datatypes
df['Adenoma_size'] = df['Adenoma_size'].astype('category')
df['Category'] = df['Category'].astype('category')
df['Patient_gender'] = df['Patient_gender'].astype('category')


## Dataframe Summary

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
print("Total Dataframe rows:", len(df))
print("Total Dataframe columns:", len(df.columns))
label_col = 'Category'
non_prol_n = len(df[df[label_col] == 'non-prolactinoma'])
prol_n = len(df[df[label_col] == 'prolactinoma'])
print("Dataframe rows with prolatinom label:", prol_n)
print("Dataframe rows with non-prolatinom label:", non_prol_n)

## Distribution Analysis
we will take a look at the distributions of the variables and check for outliers as well.

### Data of MRI
First we will take a look at the distribution of the data for the MRI which were done. 

In [ ]:
column = 'Date_Case'
print("Range of Case dates:", df[column].min().strftime('%d.%m.%Y'), "to", df[column].max().strftime('%d.%m.%Y'))
print("Missing values:", sum(df[column].isna()))

In [ ]:
sns.histplot(df[column],bins=50)
plt.title(f"Distribution of Date of Cases, n={len(df[column])}")
plt.xlabel('Date of Case')
plt.show()

### Data Features (screening data)

#### Adenoma Size
The column for the 'Adenoma_size' describes if an Adenoma was labeled as micro or macro.
Micro is defined as <10mm and a macro is defined as >= 10 mm in size.

In [ ]:
column= 'Adenoma_size'
print(f"Summary Statistics:\n{df[column].describe()}")
print(f"Percentage Distribution:\n{df[column].value_counts(normalize=True) * 100}")
print(f"Missing values Adenoma size:{sum(df[column].isna())}")

In [ ]:
sns.histplot(df[column])
plt.title(f"Distribution of Adenoma size categories, n={len(df[column])}")
plt.xlabel('Adenoma size category')
plt.show()

#### Pre Operation hormonal disfunctions
The column for the 'Ausfälle prä' describes if a patient was experiencing an under- or an overproduction of a type of hormone (produced by the pituitary gland) before an operation.
Values which indicate a prolactinomaa are an overproduction of prolactin (hyperprolaktin).


In [ ]:
# define all "pre op" columns
pre_op_columns = [col for col in df.columns if "Pre_" in col]

In [ ]:
# summarise and sort the pre op column values
summary=df[pre_op_columns].sum().sort_values(ascending=False)
sns.barplot(x=summary.index, y=summary.values)
plt.title(f"Distribution of hormonal disfunctions, multiples per patient possible, n={sum(summary)}")
plt.xlabel("Pre OP hormonal disfunctions")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

#### Date of Entry and Operation
The columns for the 'Eintrittsdatum' and 'Operationdatum' describe when a patient entered the hospital and also when the operation took place.
The difference between these values can show the significance or the extraordinary need for an operation.

In [ ]:
column = 'Diagnosis_Date'
print("Range of Diagnosis date:", df[column].min().strftime('%d.%m.%Y'), "to", df[column].max().strftime('%d.%m.%Y'))
print("Missing values:", sum(df[column].isna()))

In [ ]:
sns.histplot(df[column])
plt.title(f"Distribution of Date of Patient Diagnosis, n={len(df[column])}")
plt.xlabel('Date of Patient Diagnosis')
plt.show()

In [ ]:
column = 'Operation_date'
print("Range of Operation date:", df[column].min().strftime('%d.%m.%Y'), "to", df[column].max().strftime('%d.%m.%Y'))
print("Missing values:", sum(df[column].isna()))

In [ ]:
sns.histplot(df[column])
plt.title(f"Distribution of Date of Patient Operation, n={len(df[column])}")
plt.xlabel('Date of Patient Operation')
plt.show()

In [ ]:
# calculate time difference in years between operation and entry date
df['EDDate_OPDate_Difference'] = (df['Operation_date'] - df['Diagnosis_Date']).dt.days /365
# Create a histogram to visualize the time differences
sns.kdeplot(df['EDDate_OPDate_Difference'], fill=True)
plt.title(f"Time Difference between Entry Date and Operation Date, n={len(df[column])}")
plt.xlabel("Time Difference (years)")
plt.ylabel("Count")
plt.show()

In [ ]:
sns.stripplot(y=df['EDDate_OPDate_Difference'], jitter=True, legend=False,alpha=0.7,label="Patients")
sns.boxplot(y=df['EDDate_OPDate_Difference'], width=0.3)
plt.title(f"Scatterplot with Boxplot for a Time difference between OP Date and Entry Date, n={len(df[column])}")
plt.ylabel("Time Difference (years)")
plt.show()

#### Patient gender
The Patients gender is in the data set. It might be needed to impute certain missing values of hormone levels.

In [ ]:
column = 'Patient_gender'
print("Summary Statistics Patient Gender:\n", df[column].describe())
print("Missing values:", sum(df[column].isna()))
sns.histplot(df[column],bins=30)
plt.title(f"Distribution of Patient gender, n={len(df[column])}",fontsize=20)
plt.xlabel('Patient gender')
plt.ylabel('Count')

plt.xticks([0, 1], ['Female', 'Male'])  # Replace 0 and 1 with Female and Male
plt.show()

#### Patient Age
The Patients age is also the data set. It might be needed to impute certain missing values of hormone levels.


In [ ]:
column = 'Patient_age'
print("Range of ages:", df[column].min(), "to", df[column].max())
print("Mean Patient Age:", df[column].mean())
print("Median Patient Age:", df[column].median())
print("Missing values Patient Age:", sum(df[column].isna()))
sns.kdeplot(df[column], fill=True)
plt.title(f"Distribution of Patient age, n={len(df[column])}")
plt.xlabel('Patient age')
plt.show()

sns.stripplot(y=df[column],jitter=True, legend=False,alpha=0.7,label="Patients")
sns.boxplot(y=df[column], width=0.3)
plt.title(f"Scatterplot with Boxplot for Patient Age, n={len(df[column])}")
plt.ylabel("Patient Age")
plt.legend()
plt.show()

### Data Laboratory Data (hormonal data)

Some Patients are missing lab values because they were transferred from external facilities like Kantonsspital Baden (KSB).
If the data was found by the labelers, we can include it in the models.

#### Prolactin (hormone)
The Column 'Prolactin' contains the measured values of prolactin for the patient.


In [ ]:
column = 'PROL'
label_col = 'Category'
print("Range of Prolactin:", df[column].min(), "to", df[column].max())
print("Mean Prolactin:", df[column].mean())
print("Median Prolactin:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column], fill=True)
plt.title(f"Distribution of {column}, n={len(df[column])}")
plt.xlabel('Prolactin (μg/l)')
plt.xlim(0, None)
plt.show()

In [ ]:
print("Range of Prolactin for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean Prolactin for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median Prolactin for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of Prolactin for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean Prolactin for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median Prolactin for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))
sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('Prolactin (μg/l)')
plt.xlim(0, None)
plt.show()

#### Cortisol
The Column 'COR' contains the measured values of Cortisol for the patient.


In [ ]:
column = 'COR'
print("Range of Cortisol:", df[column].min(), "to", df[column].max())
print("Mean Cortisol:", df[column].mean())
print("Median Cortisol:", df[column].median())
print("Missing values Cortisol:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True)
plt.title(f"Distribution of {column}, n={len(df[column])}")
plt.xlabel('Cortisol (mU/l)')
plt.xlim(0, None)
plt.show()

In [ ]:
print("Range of Cortisol for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean Cortisol for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median Cortisol for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values Cortisol for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of Cortisol for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean Cortisol for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median Cortisol for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values Cortisol for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))

sns.kdeplot(df,x=column,hue=label_col,fill=True)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('Cortisol (mU/l)')
plt.xlim(0, None)

plt.show()

#### IGF1 (hormone)
The Column 'IGF1' contains the measured values of Insulin-like growth factor 1 for the patient.


In [ ]:
column = 'IGF1'
print("Range of IGF1:", df[column].min(), "to", df[column].max())
print("Mean IGF1:", df[column].mean())
print("Median IGF1:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True,)
plt.title(f"Distribution of {column}, n={len(df[column])}")
plt.xlabel('IGF1 (μg/l)')
plt.xlim(0, None)
plt.show()

In [ ]:
print("Range of IGF1 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean IGF1 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median IGF1 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values IGF1 for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of IGF1 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean IGF1 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median IGF1 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values IGF1 for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))

sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('IGF1 (μg/l)')
plt.xlim(0, None)
plt.show()

#### FT4 (hormone)
The Column 'FT4' contains the measured values of Thyroxine free hormone for the patient.


In [ ]:
column = 'FT4'
print("Range of FT4:", df[column].min(), "to", df[column].max())
print("Mean FT4:", df[column].mean())
print("Median FT4:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True,)
plt.title(f"Distribution of {column}, n={len(df[column])}")
plt.xlabel('FT4 pmol/l')
plt.show()

In [ ]:
print("Range of FT4 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean FT4 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median FT4 for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values FT4 for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of FT4 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean FT4 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median FT4 for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values FT4 for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))
sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('FT4 pmol/l')
plt.xlim(0, None)
plt.show()


#### Testosteron (hormone)
The Column 'Testosteron' contains the measured values of Testosteron for the patient.


In [ ]:
column = 'TEST'
print("Range of Testosteron:", df[column].min(), "to", df[column].max())
print("Mean Testosteron:", df[column].mean())
print("Median Testosteron:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True,)
plt.title(f"Distribution of {column}")
plt.xlabel('Testosteron nmol/l ')
plt.show()

In [ ]:
print("Range of Testosteron for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean Testosteron for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median Testosteron for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values Testosteron for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of Testosteron for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean Testosteron for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median Testosteron for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values Testosteron for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))
sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('Testosteron nmol/l')
plt.xlim(0, None)
plt.show()

#### LH (hormone)
The Column 'LH' contains the measured values of Luteinizing Hormone for the patient.


In [ ]:
column = 'LH'
print("Range of LH:", df[column].min(), "to", df[column].max())
print("Mean LH:", df[column].mean())
print("Median LH:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True,)
plt.title(f"Distribution of {column}")
plt.xlabel('LH U/l')
plt.show()

In [ ]:
print("Range of LH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean LH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median LH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values LH for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of LH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean LH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median LH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values LH for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))
sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('LH U/l')
plt.xlim(0, None)
plt.show()

#### FSH (hormone)
The Column 'FSH' contains the measured values of Follicle-stimulating hormone  for the patient.


In [ ]:
column = 'FSH'
print("Range of FSH:", df[column].min(), "to", df[column].max())
print("Mean FSH:", df[column].mean())
print("Median FSH:", df[column].median())
print("Missing values:", sum(df[column].isna()))
sns.kdeplot(df[column],fill=True,)
plt.title(f"Distribution of {column}")
plt.xlabel('FSH U/l')
plt.show()

In [ ]:
print("Range of FSH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].min(), "to", df[df[label_col] == 'prolactinoma'][column].max())
print("Mean FSH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].mean())
print("Median FSH for 'prolactinoma':", df[df[label_col] == 'prolactinoma'][column].median())
print("Missing values FSH for 'prolactinoma':", sum(df[df[label_col] == 'prolactinoma'][column].isna()))
print('\n')
print("Range of FSH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].min(), "to", df[df[label_col] == 'non-prolactinoma'][column].max())
print("Mean FSH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].mean())
print("Median FSH for 'non-prolactinoma':", df[df[label_col] == 'non-prolactinoma'][column].median())
print("Missing values FSH for 'non-prolactinoma':", sum(df[df[label_col] == 'non-prolactinoma'][column].isna()))
sns.kdeplot(df,x=column,hue=label_col,fill=True,)
plt.title(f"N(non-prolactinoma) = {non_prol_n}, N(prolactinoma) = {prol_n} ")
plt.suptitle(f'Distribution of {column}')
plt.xlabel('FSH U/l')
plt.xlim(0, None)
plt.show()

### Data Labels (Medical diagnosis data)

#### Category Prolactinoma (binary Label)
The Column 'Category' contains the actual label data if a adenoma is a prolactinoma or a non-prolactinoma.


In [ ]:
column = 'Category'
print("Summary Statistics:\n",df[column].describe())
print("Percentage Distribution:\n",df[column].value_counts(normalize=True) * 100)
print("Missing values Adenoma category:", sum(df[column].isna()))
sns.histplot(df[column])
plt.title(f"Distribution of Adenoma Category (binary Label for classification)")
plt.xlabel('Category of Adenoma')
plt.show()

In [ ]:
# Create a figure and axis (subplots)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# First subplot: Dodge plot
sns.histplot(df, x='Category', hue='Patient_gender', multiple='dodge', ax=axes[0])

# Set the title and labels for the first subplot
axes[0].set_title("Absolute Values")
axes[0].set_xlabel('Category of Adenoma')
axes[0].set_xticks(range(len(df['Category'].unique())))
axes[0].set_xticklabels(['prolactinoma','non-prolactinoma'], ha='center')

# Second subplot: Fill plot
sns.histplot(df, x='Category', hue='Patient_gender', multiple='fill', ax=axes[1])

# Set the title and labels for the second subplot
axes[1].set_title("Relative Values")
axes[1].set_xlabel('Category of Adenoma')
axes[1].set_xticks(range(len(df['Category'].unique())))
axes[1].set_xticklabels(['prolactinoma','non-prolactinoma'], ha='center')

# Adjust layout for better spacing
fig.suptitle("Comparison of Adenoma Category Distribution by Gender")
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
age_bins = [17,39,61, float('inf')]  # Define your age ranges
age_labels = ['17 - 39 years','40 - 61 years ','62+ years']  # Labels for the age ranges

# Create a new column 'AgeRange' in your DataFrame based on the age bins
df['AgeRange'] = pd.cut(df['Patient_age'], bins=age_bins, labels=age_labels, right=False)
# Create a figure and axis (subplots)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# First subplot: Dodge plot
sns.histplot(df, x='Category', hue='AgeRange', multiple='dodge', ax=axes[0])

# Set the title and labels for the first subplot
axes[0].set_title("Absolute Values")
axes[0].set_xlabel('Category of Adenoma')

# Second subplot: Fill plot
sns.histplot(df, x='Category', hue='AgeRange', multiple='fill', ax=axes[1])

# Set the title and labels for the second subplot
axes[1].set_title("Relative Values")
axes[1].set_xlabel('Category of Adenoma')

# Adjust layout for better spacing
fig.suptitle("Comparison of Adenoma Category Distribution by Age Range")
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a figure with two subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Plot the first histogram on the first subplot
sns.histplot(data=df, x='fold', ax=axes[0],bins=len(df['fold'].unique()))
axes[0].set_title('Histogram of fold')

# Plot the second histogram on the second subplot
sns.histplot(data=df, x='fold_learning_curve', ax=axes[1],bins=len(df['fold_learning_curve'].unique()))
axes[1].set_title('Histogram of fold_learning_curve')

# Adjust layout to prevent overlap of subplots
plt.tight_layout()

# Show the plot
plt.show()


# Conclusion EDA
In summary, our exploration of the small dataset, consisting of approximately 200 observations with 9 features encompassing hormones, gender, and age, has provided valuable insights, while also revealing areas that warrant careful consideration in future research endeavors.

 
The small size of the dataset underscores the potential benefits of incorporating more features and observations to bolster the robustness of any predictive model, especially deep learning models. While the option of increasing observations exists by incorporating lab data without corresponding MRI results <b> (und auch umgekehrt MRI Data without corresponding Lab Data...), </b>this approach introduces a trade-off in data quality. Extracting more features from MRI images is an avenue for consideration but given our commitment to developing multimodal models that combine tabular lab features with MRI picture data we choose not to do it. 
<b>(Daten extrahieren von MRI-Images und dann zusammenfügen ist auch multimodal!!, vielleicht eher multimodalen fusion Modellen ?)</b>
 
Examining feature distributions, we observed generally well-distributed patterns, with the highest sparsity (~45%) observed in the testosterone hormone. This sparsity is logically explained by the absence of testosterone testing in females. Addressing missing values appropriately is crucial to maintaining the integrity of our analyses. <b>(Hier noch erklären, was die Möglichkeiten sind...)</b>

Analyzing differences in hormone distributions associated with the target label, we identified notable associations. FSH and LH gender hormones exhibited lower values for prolactinoma cases, while the prolactin hormone displayed a wider distribution, notably higher in cases with prolactinoma. These distinctions, grounded in medical rationale, provide meaningful insights for feature selection in our tabular model development.

Furthermore, our analysis revealed intriguing patterns concerning patient age and gender in relation to the target label. Males demonstrated a slight (~5%) prevalence for prolactinomas. Additionally, age-related trends indicated a slight prevalence among younger individuals (17-39) and middle-aged individuals (40-61), while older individuals (62+) showed less prolactinomas, potentially indicating the occurrence of other types of brain tumors.

In conclusion, our exploratory analysis serves as a foundation for future investigations. The identified areas for improvement, such as the need for more features and observations, underscore the ongoing evolution of our research. Balancing data quality and quantity, understanding feature distributions, and recognizing medically significant associations will guide the development of more robust multimodal models in subsequent studies.




- small dataset (~200 observations and 9 features, 7 hormones and gender as well as age)
    - more features and more observations are probably needed
    - more observations might be possible if we decrease the dataquality by using lab data which does not correspond with an MRI, especially for deep learning models
    - more features could be extracted from MRI images but for the lab data we used all available features, we will not do it because we wanted to use multimodal models (tabular lab features and mri picture data)
- all features are distributed well, the highest sparsity value ~45% is in the testosteron hormone, this can be explained by the missing values for females which testosteron is not tested in the lab
- differences in label in distributions of hormones
    - FSH and LH gender hromones are associated with lower values for a prolactinoma, prolaktine hormone is much wider distributed (aka higher values) for cases with prolactinoma
    - both of these differences are medically explainable 
- some differences in patient age and gender for label
    - males have a little ~5% prevalence
    - younger people (17-39) have a little prevalence for a prolactinoma, middle aged (40-61) also have a little prevalence, older people (62+) might have other brain tumors